# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [12]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [13]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс15.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.0,5.000000,5,0.142857,0.111111,0.142857,0.111111
1,Экспертиза промышленной безопасности,0.2,1.000000,3,0.200000,0.111111,0.200000,0.111111
2,Готовность к ликвидации последствий аварий и и...,0.2,0.333333,1,0.333333,0.111111,0.142857,0.111111
3,Оформление информационными знаками и знаками б...,7.0,5.000000,3,1.000000,1.000000,0.200000,0.111111
4,Содержание объектов и оборудования,9.0,9.000000,9,1.000000,1.000000,1.000000,1.000000


In [14]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [15]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов        int64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [16]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0            1.0                              5.000000   
1            0.2                              1.000000   
2            0.2                              0.333333   
3            7.0                              5.000000   
4            9.0                              9.000000   
5            7.0                              5.000000   
6            9.0                              0.000000   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                                  5         
1                                                  3         
2                                                  1         
3                                                  3         
4                                                  9         
5                                                  7         
6                                                  9         

   Оформление информационными знаками 

In [17]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов        int64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [18]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [19]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,1.0,5.000000,0.142857,0.111111,0.142857,0.111111
1,0.2,1.000000,0.200000,0.111111,0.200000,0.111111
2,0.2,0.333333,0.333333,0.111111,0.142857,0.111111
3,7.0,5.000000,1.000000,1.000000,0.200000,0.111111
4,9.0,9.000000,1.000000,1.000000,1.000000,1.000000
5,7.0,5.000000,5.000000,1.000000,1.000000,1.000000
6,9.0,0.000000,9.000000,1.000000,1.000000,1.000000


In [20]:
DF_1=df[:].copy()

In [21]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                 33.400000
Экспертиза промышленной безопасности                         25.333333
Готовность к ликвидации последствий аварий и инцидентов      37.000000
Оформление информационными знаками и знаками безопасности    16.676190
Содержание объектов и оборудования                            4.333333
Охранные зоны (ОЗ)                                            3.685714
Организация рабочего процесса и безопасности персонала        3.444444
dtype: float64

In [22]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [23]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.0,5.000000,5,0.142857,0.111111,0.142857,0.111111,0.029940,0.197368,0.135135,0.008567,0.025641,0.038760,0.032258
1,0.2,1.000000,3,0.200000,0.111111,0.200000,0.111111,0.005988,0.039474,0.081081,0.011993,0.025641,0.054264,0.032258
2,0.2,0.333333,1,0.333333,0.111111,0.142857,0.111111,0.005988,0.013158,0.027027,0.019989,0.025641,0.038760,0.032258
3,7.0,5.000000,3,1.000000,1.000000,0.200000,0.111111,0.209581,0.197368,0.081081,0.059966,0.230769,0.054264,0.032258
4,9.0,9.000000,9,1.000000,1.000000,1.000000,1.000000,0.269461,0.355263,0.243243,0.059966,0.230769,0.271318,0.290323


In [24]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.029940,0.197368,0.135135,0.008567,0.025641,0.038760,0.032258
1,0.005988,0.039474,0.081081,0.011993,0.025641,0.054264,0.032258
2,0.005988,0.013158,0.027027,0.019989,0.025641,0.038760,0.032258
3,0.209581,0.197368,0.081081,0.059966,0.230769,0.054264,0.032258
4,0.269461,0.355263,0.243243,0.059966,0.230769,0.271318,0.290323
5,0.209581,0.197368,0.189189,0.299829,0.230769,0.271318,0.290323
6,0.269461,0.000000,0.243243,0.539692,0.230769,0.271318,0.290323


In [25]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.029940,0.197368,0.135135,0.008567,0.025641,0.038760,0.032258,0.066810
1,0.005988,0.039474,0.081081,0.011993,0.025641,0.054264,0.032258,0.035814
2,0.005988,0.013158,0.027027,0.019989,0.025641,0.038760,0.032258,0.023260
3,0.209581,0.197368,0.081081,0.059966,0.230769,0.054264,0.032258,0.123612
4,0.269461,0.355263,0.243243,0.059966,0.230769,0.271318,0.290323,0.245763


In [26]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [27]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [28]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

0.4708857385681649
0.2485076631907512
0.18081050851330574
1.1634171907033446
2.0070718688617357
2.1781254099134215
2.673644462491972


In [29]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

8.922462842242695


In [30]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.3204104737071158
1.4142857142857144
0.22655286019695053


In [31]:
# Получилось, что значение CR=0.22, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами